# Assignment 2: Elliptic Curve Digital Signature Algorithm (EC-DSA)

## Analyzing TLS Certificate Elliptic Curve Parameters

---

## Section 1: Introduction

### What are Digital Signatures?

**Digital signatures** are cryptographic mechanisms that provide:
- **Authentication**: Verification that a message comes from the claimed sender
- **Integrity**: Assurance that the message has not been altered
- **Non-repudiation**: The sender cannot deny having sent the message

They are the digital equivalent of handwritten signatures but offer much stronger security guarantees through mathematical foundations.

### What is EC-DSA?

**Elliptic Curve Digital Signature Algorithm (EC-DSA)** is a variant of the Digital Signature Algorithm (DSA) that uses **elliptic curve cryptography** instead of traditional integer factorization or discrete logarithm problems.

**Advantages of EC-DSA:**
- **Smaller key sizes**: A 256-bit EC key provides comparable security to a 3072-bit RSA key
- **Faster computations**: Shorter keys mean faster signing and verification
- **Lower bandwidth**: Smaller signatures reduce data transmission overhead

### Role in TLS/HTTPS

In **Transport Layer Security (TLS)**, EC-DSA is used to:
1. **Authenticate the server** to the client during the handshake
2. **Sign the key exchange parameters** to prevent man-in-the-middle attacks
3. **Establish trust** through certificate chains

When you visit a website over HTTPS, the server presents a **TLS certificate** containing its public key. If the certificate uses EC-DSA, the public key is a point on an elliptic curve.

### Assignment Objective

In this assignment, we will:
1. Load and parse a real TLS certificate from `_.bits-pilani.ac.in`
2. Extract elliptic curve information from the certificate
3. Identify the finite field used
4. Understand what information is directly available vs. what requires external standards
5. Recover the complete elliptic curve equation using NIST/SEC specifications

---

## Section 2: Problem Statement

### The Core Question

> **Given a website TLS certificate that uses EC-DSA for its public key, can we extract the elliptic curve equation and the characteristic of the finite field?**

### Why This Question is Non-Trivial

This question appears simple at first glance, but it reveals important aspects of how cryptographic standards work in practice:

1. **Certificates don't store curve equations**: For efficiency and standardization, TLS certificates only store a **curve identifier** (like "secp256r1"), not the actual mathematical parameters.

2. **Named curves vs. explicit parameters**: The industry uses **named curves** — pre-defined, well-studied curves with known security properties. Storing explicit parameters would be wasteful and could introduce non-standard (potentially weak) curves.

3. **Separation of concerns**: The certificate tells us *which* curve is used; external standards (NIST, SEC) tell us *what* that curve actually is mathematically.

4. **Trust through standardization**: Using named curves ensures that all parties agree on the exact mathematical definition, preventing subtle implementation differences.

### Our Approach

We will demonstrate that:
- The **curve name** can be extracted directly from the certificate
- The **curve equation** and **field characteristic** must be looked up from standards
- This is by design, not a limitation

---

## Section 3: Certificate Overview

### What Does a TLS Certificate Contain?

A TLS certificate (X.509 format) contains:

| Field | Description |
|-------|-------------|
| **Subject** | Identity of the certificate holder (e.g., domain name) |
| **Issuer** | Certificate Authority (CA) that issued the certificate |
| **Validity Period** | Start and expiration dates |
| **Public Key** | The subject's public key for encryption/verification |
| **Signature Algorithm** | Algorithm used by the CA to sign the certificate |
| **Extensions** | Additional metadata (key usage, subject alt names, etc.) |

### Public Key Information in Certificates

For EC-DSA certificates, the public key section contains:
1. **Algorithm identifier**: Indicates this is an EC key (e.g., `id-ecPublicKey`)
2. **Named curve OID**: Object Identifier specifying which curve (e.g., `secp256r1`)
3. **Public key point**: The actual public key as a point $(Q_x, Q_y)$ on the curve

### Named Elliptic Curves

Instead of storing the full curve equation, certificates use **named curves**:

| Name | Also Known As | Field Size |
|------|---------------|------------|
| secp256r1 | prime256v1, P-256 | 256 bits |
| secp384r1 | P-384 | 384 bits |
| secp521r1 | P-521 | 521 bits |

### What is NOT Stored in Certificates

The following are **NOT** explicitly stored in the certificate:
- The prime $p$ (field characteristic)
- Curve coefficients $a$ and $b$
- Generator point $G$
- Order $n$ of the curve

These must be obtained from the standard specification (NIST FIPS 186-4 or SEC 2).

---

## Section 4: Loading and Parsing the Certificate

We will use Python's `cryptography` library to load and parse the TLS certificate exported from `_.bits-pilani.ac.in`.

In [7]:
# Import required libraries
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec

# Path to the certificate file
CERT_FILE = "_.bits-pilani.ac.in.crt"

# Load and parse the certificate
with open(CERT_FILE, "rb") as f:
    cert_data = f.read()

# Parse as X.509 certificate (PEM format)
certificate = x509.load_pem_x509_certificate(cert_data, default_backend())

print("=" * 60)
print("CERTIFICATE BASIC INFORMATION")
print("=" * 60)

# Extract and display Subject
print("\nSubject (Certificate Holder):")
print(f"   {certificate.subject.rfc4514_string()}")

# Extract and display Issuer
print("\nIssuer (Certificate Authority):")
print(f"   {certificate.issuer.rfc4514_string()}")

# Extract and display Signature Algorithm
print("\nSignature Algorithm:")
print(f"   {certificate.signature_algorithm_oid._name}")

# Display Validity Period
print("\nValidity Period:")
print(f"   Not Before: {certificate.not_valid_before_utc}")
print(f"   Not After:  {certificate.not_valid_after_utc}")

print("\n" + "=" * 60)

CERTIFICATE BASIC INFORMATION

Subject (Certificate Holder):
   CN=*.bits-pilani.ac.in

Issuer (Certificate Authority):
   CN=Sectigo ECC Domain Validation Secure Server CA,O=Sectigo Limited,L=Salford,ST=Greater Manchester,C=GB

Signature Algorithm:
   ecdsa-with-SHA256

Validity Period:
   Not Before: 2024-11-30 00:00:00+00:00
   Not After:  2025-12-31 23:59:59+00:00



### Interpretation of Certificate Fields

- **Subject**: The entity that owns this certificate (the website `_.bits-pilani.ac.in`)
- **Issuer**: The Certificate Authority (CA) that vouches for the subject's identity
- **Signature Algorithm**: The algorithm the CA used to sign this certificate
- **Validity Period**: The time window during which this certificate is considered valid

---

## Section 5: Extracting the EC Public Key

Now we extract the public key and verify that it uses elliptic curve cryptography.

In [8]:
# Extract the public key from the certificate
public_key = certificate.public_key()

print("=" * 60)
print("PUBLIC KEY ANALYSIS")
print("=" * 60)

# Check if it's an Elliptic Curve public key
if isinstance(public_key, ec.EllipticCurvePublicKey):
    print("\n[OK] Public Key Type: Elliptic Curve Public Key (EC)")
    
    # Get the curve used
    curve = public_key.curve
    curve_name = curve.name
    key_size = curve.key_size
    
    print(f"\nElliptic Curve Information:")
    print(f"   Curve Name: {curve_name}")
    print(f"   Key Size:   {key_size} bits")
    
    # Get the public key point coordinates
    public_numbers = public_key.public_numbers()
    
    print(f"\nPublic Key Point Q = (Qx, Qy):")
    print(f"   Qx = {hex(public_numbers.x)}")
    print(f"   Qy = {hex(public_numbers.y)}")
    
else:
    print("\n[ERROR] This certificate does NOT use an Elliptic Curve public key.")
    print(f"   Actual type: {type(public_key)}")

print("\n" + "=" * 60)

PUBLIC KEY ANALYSIS

[OK] Public Key Type: Elliptic Curve Public Key (EC)

Elliptic Curve Information:
   Curve Name: secp256r1
   Key Size:   256 bits

Public Key Point Q = (Qx, Qy):
   Qx = 0x7ec2730cacc0243164ad0d08634351fef839294af0d80da4ee44914470ac4c09
   Qy = 0x4c668b29156850ffcb77283d9ddf71d69d900ff4c8c2246abb456415668d1cd0



### Understanding the Extracted Information

**Curve Name**: The certificate identifies the curve by its standard name (e.g., `secp256r1`). This name is an **Object Identifier (OID)** that uniquely references a specific curve defined in standards.

**Common Curve Name Aliases:**
| Standard Name | NIST Name | OpenSSL Name |
|---------------|-----------|--------------|
| secp256r1 | P-256 | prime256v1 |
| secp384r1 | P-384 | secp384r1 |
| secp521r1 | P-521 | secp521r1 |

**Public Key Point**: The public key $Q$ is a point on the elliptic curve, represented by coordinates $(Q_x, Q_y)$. This point is computed as $Q = d \cdot G$, where $d$ is the private key and $G$ is the generator point.

**Why Only the Curve Name?**
The certificate stores only the curve name because:
1. All parties have access to the same standards
2. It reduces certificate size
3. It ensures interoperability

---

## Section 6: Finite Field Identification

Elliptic curves used in cryptography are defined over **finite fields**. There are two main types:

### Types of Finite Fields

1. **Prime Fields $\mathbb{F}_p$**: Elements are integers modulo a prime $p$
   - Used by most NIST curves (P-256, P-384, P-521)
   - Arithmetic: standard modular arithmetic
   
2. **Binary Fields $\mathbb{F}_{2^m}$**: Elements are polynomials over $\mathbb{F}_2$
   - Used by some specialized curves (e.g., sect283k1)
   - Arithmetic: polynomial arithmetic modulo an irreducible polynomial

### Identifying the Field Type

In [9]:
# Identify the finite field type based on the curve name
print("=" * 60)
print("FINITE FIELD IDENTIFICATION")
print("=" * 60)

# The curve name tells us the field type
# "sec" curves: secp = prime field, sect = binary field
# The number indicates the field size in bits

curve_name = public_key.curve.name
key_size = public_key.curve.key_size

print(f"\nCurve Name: {curve_name}")

# Determine field type from curve name
if curve_name.startswith("secp") or "prime" in curve_name.lower():
    field_type = "Prime Field (F_p)"
    field_char = "p (a large prime number)"
    print(f"\nField Type: {field_type}")
    print(f"   - This is a prime field where arithmetic is performed modulo a prime p")
    print(f"   - Field size: {key_size} bits")
    print(f"   - The characteristic of the field is p itself")
    
elif curve_name.startswith("sect"):
    field_type = "Binary Field (F_{2^m})"
    field_char = "2"
    print(f"\nField Type: {field_type}")
    print(f"   - This is a binary extension field")
    print(f"   - Field size: 2^{key_size}")
    print(f"   - The characteristic of the field is 2")

else:
    print(f"\n[WARNING] Unknown curve type: {curve_name}")

print("\n" + "=" * 60)

FINITE FIELD IDENTIFICATION

Curve Name: secp256r1

Field Type: Prime Field (F_p)
   - This is a prime field where arithmetic is performed modulo a prime p
   - Field size: 256 bits
   - The characteristic of the field is p itself



---

## Section 7: Mapping Curve Name to Mathematical Parameters

### ⚠️ Important Clarification

**The following parameters are NOT extracted from the certificate directly.**

The certificate only contains the **curve name** (e.g., `secp256r1`). The actual mathematical parameters must be obtained from **public standards**:
- **NIST FIPS 186-4**: Digital Signature Standard
- **SEC 2**: Recommended Elliptic Curve Domain Parameters

This is by design — using standardized, well-analyzed curves ensures security and interoperability.

### Standard Curve Parameters Lookup

In [10]:
# Standard elliptic curve parameters from NIST/SEC specifications
# These are NOT from the certificate - they are from public standards

CURVE_PARAMETERS = {
    "secp256r1": {
        "aliases": ["prime256v1", "P-256"],
        "field_type": "Prime Field",
        # Prime p (field characteristic)
        "p": 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF,
        # Curve coefficient a
        "a": 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFC,
        # Curve coefficient b
        "b": 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B,
        # Generator point G (x-coordinate)
        "Gx": 0x6B17D1F2E12C4247F8BCE6E563A440F277037D812DEB33A0F4A13945D898C296,
        # Generator point G (y-coordinate)
        "Gy": 0x4FE342E2FE1A7F9B8EE7EB4A7C0F9E162BCE33576B315ECECBB6406837BF51F5,
        # Order of the curve (number of points)
        "n": 0xFFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551,
        # Cofactor
        "h": 1
    },
    "secp384r1": {
        "aliases": ["P-384"],
        "field_type": "Prime Field",
        "p": 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFF0000000000000000FFFFFFFF,
        "a": 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFF0000000000000000FFFFFFFC,
        "b": 0xB3312FA7E23EE7E4988E056BE3F82D19181D9C6EFE8141120314088F5013875AC656398D8A2ED19D2A85C8EDD3EC2AEF,
        "Gx": 0xAA87CA22BE8B05378EB1C71EF320AD746E1D3B628BA79B9859F741E082542A385502F25DBF55296C3A545E3872760AB7,
        "Gy": 0x3617DE4A96262C6F5D9E98BF9292DC29F8F41DBD289A147CE9DA3113B5F0B8C00A60B1CE1D7E819D7A431D7C90EA0E5F,
        "n": 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFC7634D81F4372DDF581A0DB248B0A77AECEC196ACCC52973,
        "h": 1
    },
    "secp521r1": {
        "aliases": ["P-521"],
        "field_type": "Prime Field",
        "p": 0x01FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,
        "a": 0x01FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFC,
        "b": 0x0051953EB9618E1C9A1F929A21A0B68540EEA2DA725B99B315F3B8B489918EF109E156193951EC7E937B1652C0BD3BB1BF073573DF883D2C34F1EF451FD46B503F00,
        "Gx": 0x00C6858E06B70404E9CD9E3ECB662395B4429C648139053FB521F828AF606B4D3DBAA14B5E77EFE75928FE1DC127A2FFA8DE3348B3C1856A429BF97E7E31C2E5BD66,
        "Gy": 0x011839296A789A3BC0045C8A5FB42C7D1BD998F54449579B446817AFBD17273E662C97EE72995EF42640C550B9013FAD0761353C7086A272C24088BE94769FD16650,
        "n": 0x01FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFA51868783BF2F966B7FCC0148F709A5D03BB5C9B8899C47AEBB6FB71E91386409,
        "h": 1
    }
}

# Look up parameters for the extracted curve
curve_name = public_key.curve.name

print("=" * 70)
print("ELLIPTIC CURVE PARAMETERS (FROM STANDARDS)")
print("=" * 70)
print("\nNOTE: These parameters are from NIST/SEC standards,")
print("      NOT extracted from the certificate.\n")

if curve_name in CURVE_PARAMETERS:
    params = CURVE_PARAMETERS[curve_name]
    
    print(f"Curve: {curve_name}")
    print(f"   Aliases: {', '.join(params['aliases'])}")
    print(f"   Field Type: {params['field_type']}")
    
    print(f"\nPrime p (Field Characteristic):")
    print(f"   p = {hex(params['p'])}")
    print(f"   (Decimal: {params['p']})")
    print(f"   Bit length: {params['p'].bit_length()} bits")
    
    print(f"\nCurve Coefficients:")
    print(f"   a = {hex(params['a'])}")
    print(f"   b = {hex(params['b'])}")
    
    print(f"\nElliptic Curve Equation:")
    print(f"   y^2 = x^3 + ax + b (mod p)")
    print(f"\n   Substituting values:")
    print(f"   y^2 = x^3 + ({hex(params['a'])})x")
    print(f"        + ({hex(params['b'])})")
    print(f"        (mod {hex(params['p'])})")
    
    print(f"\nGenerator Point G:")
    print(f"   Gx = {hex(params['Gx'])}")
    print(f"   Gy = {hex(params['Gy'])}")
    
    print(f"\nCurve Order:")
    print(f"   n = {hex(params['n'])}")
    print(f"   Cofactor h = {params['h']}")
    
else:
    print(f"[WARNING] Parameters for curve '{curve_name}' not in our lookup table.")
    print("   Please refer to SEC 2 or NIST FIPS 186-4 for parameters.")

print("\n" + "=" * 70)

ELLIPTIC CURVE PARAMETERS (FROM STANDARDS)

NOTE: These parameters are from NIST/SEC standards,
      NOT extracted from the certificate.

Curve: secp256r1
   Aliases: prime256v1, P-256
   Field Type: Prime Field

Prime p (Field Characteristic):
   p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
   (Decimal: 115792089210356248762697446949407573530086143415290314195533631308867097853951)
   Bit length: 256 bits

Curve Coefficients:
   a = 0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc
   b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b

Elliptic Curve Equation:
   y^2 = x^3 + ax + b (mod p)

   Substituting values:
   y^2 = x^3 + (0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc)x
        + (0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b)
        (mod 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff)

Generator Point G:
   Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d

### Verifying the Public Key Lies on the Curve

As a sanity check, we can verify that the public key point $(Q_x, Q_y)$ satisfies the curve equation.

In [11]:
# Verify that the public key point lies on the curve
# This confirms that our looked-up parameters are correct

if curve_name in CURVE_PARAMETERS:
    params = CURVE_PARAMETERS[curve_name]
    
    # Get public key coordinates
    Qx = public_numbers.x
    Qy = public_numbers.y
    
    # Get curve parameters
    p = params['p']
    a = params['a']
    b = params['b']
    
    # Compute left side: y^2
    left_side = pow(Qy, 2, p)
    
    # Compute right side: x^3 + ax + b
    right_side = (pow(Qx, 3, p) + a * Qx + b) % p
    
    print("=" * 60)
    print("PUBLIC KEY VERIFICATION")
    print("=" * 60)
    print("\nChecking if Q = (Qx, Qy) satisfies y^2 = x^3 + ax + b (mod p)")
    print(f"\n   Left side  (y^2 mod p):      {left_side}")
    print(f"   Right side (x^3+ax+b mod p): {right_side}")
    
    if left_side == right_side:
        print("\n   [VERIFIED] The public key point lies on the curve.")
    else:
        print("\n   [ERROR] The point does NOT lie on the curve.")
    
    print("\n" + "=" * 60)

PUBLIC KEY VERIFICATION

Checking if Q = (Qx, Qy) satisfies y^2 = x^3 + ax + b (mod p)

   Left side  (y^2 mod p):      48865865149252745848206311639282198504487504313120810187220122553894272281691
   Right side (x^3+ax+b mod p): 48865865149252745848206311639282198504487504313120810187220122553894272281691

   [VERIFIED] The public key point lies on the curve.



---

## Section 8: Final Answer to the Key Question

### Can we extract the elliptic curve equation from the certificate?

**No, we cannot extract the elliptic curve equation directly from the certificate.**

### What CAN we extract from the certificate?

| Information | Extractable? | Source |
|-------------|--------------|--------|
| Curve name (e.g., secp256r1) | ✅ Yes | Certificate |
| Public key type (EC) | ✅ Yes | Certificate |
| Public key point (Qx, Qy) | ✅ Yes | Certificate |
| Key size (e.g., 256 bits) | ✅ Yes | Certificate |
| Prime p | ❌ No | Standards |
| Coefficients a, b | ❌ No | Standards |
| Generator point G | ❌ No | Standards |
| Curve order n | ❌ No | Standards |

### How do we obtain the curve equation?

1. **Extract the curve name** from the certificate (e.g., `secp256r1`)
2. **Look up the parameters** in published standards:
   - NIST FIPS 186-4 (Digital Signature Standard)
   - SEC 2 (Standards for Efficient Cryptography)
3. **Construct the equation**: $y^2 \equiv x^3 + ax + b \pmod{p}$

### Why is this design appropriate?

1. **Efficiency**: Storing just a curve identifier is much smaller than storing all parameters
2. **Security**: Named curves have been thoroughly analyzed for cryptographic strength
3. **Interoperability**: All implementations can agree on the exact parameters
4. **Trust**: Using non-standard curves would raise security concerns

### Summary

The certificate serves as a **reference** to the curve, not a complete mathematical specification. The actual elliptic curve parameters are defined in international standards and must be retrieved from those authoritative sources.

---

## Section 9: References

### Standards Documents

1. **NIST FIPS 186-4**: Digital Signature Standard (DSS)
   - URL: https://csrc.nist.gov/publications/detail/fips/186/4/final
   - Defines ECDSA and recommended curves (P-256, P-384, P-521)

2. **SEC 2**: Recommended Elliptic Curve Domain Parameters
   - URL: https://www.secg.org/sec2-v2.pdf
   - Comprehensive specification of secp curves

3. **RFC 5480**: Elliptic Curve Cryptography Subject Public Key Information
   - URL: https://tools.ietf.org/html/rfc5480
   - Defines how EC keys are encoded in X.509 certificates

### Python Libraries

4. **Python `cryptography` Library Documentation**
   - URL: https://cryptography.io/en/latest/
   - Used for certificate parsing and EC key handling

### Additional Reading

5. **Elliptic Curve Cryptography (ECC)** - Wikipedia
   - URL: https://en.wikipedia.org/wiki/Elliptic-curve_cryptography

6. **X.509 Certificate Standard** - ITU-T
   - URL: https://www.itu.int/rec/T-REC-X.509

---

*Assignment 2 - Cryptography Course*  
*Analysis of EC-DSA in TLS Certificates*